In [1]:
import os
from os.path import join
import pandas as pd
import subprocess
import shutil
from tqdm import tqdm

##### Change this:

In [2]:
delta_G_file = r"Q:\MK\ag_wagner\Current lab members\Sarah\11 - Writing\µLife Review_T4SS-secreted TMEs\TMEs_topology prediction\xxLegionella pneumophila\Legionella-strains_FASTA\Legionella_pneumophila_str._Philadelphia_1_effector_proteins_FASTA_18-35.xlsx"
deep_file = r"Q:\MK\ag_wagner\Current lab members\Sarah\11 - Writing\µLife Review_T4SS-secreted TMEs\TMEs_topology prediction\xxLegionella pneumophila\Legionella-strains_FASTA\Legionella_pneumophila_str._Philadelphia_1_effector_proteins_Deep-Output.tsv"

In [3]:
# Define names of runs
names = ["dG", "deep_preds"]
# Path to ipredEMC
ipredEMC_folder = r"Q:\MK\ag_wagner\Current lab members\Sarah\11 - Writing\µLife Review_T4SS-secreted TMEs\TMEs_topology prediction\Scripts_TME-Topology\ipredEMC"

## Prepare directory structure

In [4]:
fastas_dir = join(ipredEMC_folder, "fastas")
predictions_dir = join(ipredEMC_folder, "predictions")
results_dir = join(ipredEMC_folder, "results")
data_dir = join(ipredEMC_folder, "data")

if not os.path.isfile(delta_G_file):
    delta_G_file = join(data_dir, delta_G_file)

# Remove old fasta files
if os.path.exists(fastas_dir):
    shutil.rmtree(fastas_dir)
os.makedirs(fastas_dir, exist_ok=True)
os.makedirs(predictions_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

## Helper methods

In [5]:
def parse_list(list_string: str) -> list:
    list_string = list_string[1:-1].replace("'", "")
    if list_string and list_string != "":
        return list_string.split(", ")
    else:
        return []

def select_positions(list: list, positions: list) -> list:
    return [list[pos] for pos in positions]

In [6]:
summary_table = pd.read_excel(join(data_dir, "TME-Analysis_Legionella-spp.xlsx"), sheet_name="Philadelphia 1", skiprows=4)
def row_by_refseq(refseq: str) -> pd.DataFrame:
    return summary_table.loc[summary_table["RefSeq"] == refseq]


In [7]:
info_map = {}
def extend_info_map(idx:str , sequence: str, length, protein):
    info_map[idx] = {
        "sequence (N->C)": sequence,
        "length": length,
        "Gene name": "/".join(protein["Gene name"].values),
        "Protein name": "/".join(protein["Protein name"].values),
        "RefSeq": protein["RefSeq"].values[0],
        "UniProt ID": protein["UniProt ID"].values[0],
    }

def write_fasta(out_path, idx, sequence):
    with open(out_path, "a") as outfile:
        outfile.write(f"{idx}\n")
        outfile.write(f"{sequence}\n")

## Fastas

### Extract fasta from DeltaG predictions

In [8]:
def extract_fasta_dG(in_path:str, out_path:str=join(fastas_dir, "dG.fasta")):
    dG_preds = pd.read_excel(in_path, 0, index_col=0).dropna(axis="rows", subset=["M sequences"])
    for i, row in tqdm(dG_preds.iterrows()):
        sequences = parse_list(row["M sequences"])
        if sequences and len(sequences) > 0:
            lengths = parse_list(row["M lengths"])
            refseq = str(row["name"]).split("|")[3]
            protein = row_by_refseq(refseq)

            domain_counter = 1
            for i, sequence in enumerate(sequences):
                idx = f"{row["name"]} - Domain {domain_counter}"

                extend_info_map(idx, sequence, lengths[i], protein)

                write_fasta(out_path, idx, sequence)
                domain_counter += 1

In [9]:
extract_fasta_dG(in_path = delta_G_file,out_path = join(fastas_dir, "dG.fasta"))

156it [00:05, 30.11it/s]


### Extract fasta from Deep predictions

In [10]:
def extract_fasta_deep(in_path:str, out_path:str=join(fastas_dir, "deep_preds.fasta")):
    deep_preds = pd.read_csv(in_path, sep="\t", index_col=0)
    for i, row in tqdm(deep_preds.iterrows()):
        letters = parse_list(row["Letter"])
        m_positions = [i for i, letter in enumerate(letters) if letter == "M"]
        if m_positions:
            sequences = parse_list(row["aminoacids"])
            lengths = parse_list(row["length"])
            refseq_id = row["Uniprot-ID"]
            protein = row_by_refseq(refseq_id)
            
            domain_counter = 1
            for i in m_positions:
                idx = f">{row.name} - Domain {domain_counter}"
                
                extend_info_map(idx, sequences[i], lengths[i], protein)

                write_fasta(out_path, idx, sequences[i])
                domain_counter += 1

In [11]:
extract_fasta_deep(in_path = deep_file, out_path = join(fastas_dir, "deep_preds.fasta"))

296it [00:02, 127.71it/s]


### Bring fasta sequences into right order

In [12]:
def turn_selected_fasta_sequences(in_path:str):
    """
    Convert sequence order from "N->C" to "Exo->Cyto"
    """
    all_affinities_df = pd.read_excel(in_path, sheet_name=0)
    for name in names:
        fasta_path = join(fastas_dir, f"{name}.fasta")
        backup_fasta_path = join(fastas_dir, f"{name}.bak")
        shutil.move(fasta_path, backup_fasta_path)

        with open(backup_fasta_path, 'r') as fasta_in:
            lines = fasta_in.readlines()
        for i, line in enumerate(lines):
            if line.startswith(">"):
                header = line[:-1]
                sequence = lines[i+1][:-1]
                split = header.split(" - ")
                domain = " " + split[-1]
                search_header = " - ".join(split[:-1]) + " "

                # Turn the sequence if the header and domain-specific orientation is given as "turn"
                if (all_affinities_df[(all_affinities_df["FASTA"] == search_header) & (all_affinities_df["TMS"] == domain)]["Orientation"] == "turn").all():
                    sequence = list(sequence)
                    sequence.reverse()
                    sequence = "".join(sequence)

                # Write to fasta and results table (=info_map)
                info_map[header]["sequence (Exo->Cyt)"] = sequence
                write_fasta(fasta_path, header, sequence)

In [13]:
turn_selected_fasta_sequences(
    in_path = join(data_dir, "ipredEMC_affinities_all-results.xlsx")
)

c:\Users\mktrens1\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\mktrens1\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


## Prediction

### Run EMC predictions

In [16]:
def emc_prediction(file_name:str):
    process = subprocess.Popen(
        ["uv", "run", "python", "prediction.py", os.path.join("fastas", f"{file_name}.fasta")],
        cwd="..", stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    process.wait()
    shutil.move(os.path.join(ipredEMC_folder, "predicted_affinity.csv"), os.path.join(predictions_dir, f"{file_name}_affinities.tsv"))

In [17]:
for name in tqdm(names):
    emc_prediction(name)

100%|██████████| 2/2 [01:00<00:00, 30.22s/it]


In [18]:
# Use this if you want to predict the EMCs in another FASTA file (hast to be in the "fastas" folder in this directory here)
# emc_prediction("TMDs-of-F-and-G-3_Sophie")

## Collect Results

In [19]:
def collect_results(file_names:str):
    for file_name in file_names:
        predictions = pd.read_csv(os.path.join(predictions_dir, f"{file_name}_affinities.tsv"), sep="\t")
        for i, row in predictions.iterrows():
            idx = f">{row["TMD"]}"
            affinity = row["Affinity to EMC"]
            info_map[idx].update({"Affinity to EMC": affinity})

    results = pd.DataFrame(info_map).transpose()
    results.to_csv(os.path.join(results_dir, "results.csv"))

In [20]:
collect_results(names)